# Segmenting and Clustering Neighborhoods in Toronto

## To convert the table listing the postal codes of Canada to a dataframe, I have used pandas library

In [ ]:
import pandas as pd

# This Reads all the dataframes from the html contents and stores them as a list. 
# Here we are reading all the dataframes from the wikipedia URL

list_of_dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
list_of_dfs

## The postal codes dataframe is identified to be avaialbe at index = 0

In [2]:
df = list_of_dfs[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


# After identifying and obtainng the correct dataframe for postal codes of Canada, we have to move the first row as column header

In [3]:
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


# Only process the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned.

In [4]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


# If more than one neighborhood exist in one postal code area then these two rows will be combined into one row with the neighborhoods separated with a comma

In [5]:
df = df.groupby(['Postcode', 'Borough'], as_index=False)['Neighbourhood'].apply(','.join).reset_index().rename(columns={0:'Neighbourhood'})
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# If a cell has a borough but a Not assigned neighborhood, then the Neighborhood will be same as the Borough

In [6]:
df['Neighbourhood'] = df.apply(lambda x: x['Borough'] if x['Neighbourhood'] == 'Not assigned' else x['Neighbourhood'], axis=1)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [7]:
df.shape

(103, 3)

# Download csv file that has the geographical coordinates of each postal code, read the csv file to a dataframe

#### Here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [8]:
df_gc = pd.read_csv('Geospatial_Coordinates.csv')
df_gc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merge the two dataframes to get a single dataframe consisting of Longitudes and Latitudes for all the Postalcodes. Before merging check the data type and column name to merge and make sure they match. 

In [9]:
print('Data type of Postal code df \n', df.dtypes)
print ()
print('Data types of GeoCode df \n', df_gc.dtypes)

Data type of Postal code df 
 Postcode         object
Borough          object
Neighbourhood    object
dtype: object

Data types of GeoCode df 
 Postal Code     object
Latitude       float64
Longitude      float64
dtype: object


In [10]:
# Before Merging rename the Postal Code in df_gc to Postcode to match with the df
df_gc.rename(columns={'Postal Code':'Postcode'}, inplace = True)
df_gc.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# After merge check the shape. This is the final DataFrame.

In [11]:
#df_result = pd.merge(df, df_gc[['Latitude', 'Longitude']], left_on = 'Postcode', right_on = 'Postcode', how = 'inner
df_result = pd.DataFrame(pd.merge(df.assign(Postcode=df.Postcode.astype(str)), df_gc.assign(Postcode=df_gc.Postcode.astype(str)),how='inner', on='Postcode'))

In [13]:
df_result.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [14]:
df_result.shape

(103, 5)

## Map and Segment and Cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [69]:
toronto_data = df_result[df_result['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## Get the Latitude and Longitude of Toronto

In [16]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto City, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7189883, -79.44157.


## Create a map of Toronto with neighborhoods

In [20]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library


# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Let's explore one of the neighborhood in Central Toronto from the Toronto dataframe

In [22]:
toronto_data.loc[4, 'Neighbourhood']

'Lawrence Park'

#### Get the neighborhood's latitude and longitude values

In [24]:
neighborhood_latitude = toronto_data.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[4, 'Neighbourhood'] # neighborhood name


print('Latitude and longitude values of {} are {}, {}.'.format(toronto_data.loc[4, 'Neighbourhood'], 
                                                               toronto_data.loc[4, 'Latitude'], 
                                                               toronto_data.loc[4, 'Longitude']))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


### Now, let's get the top 10 venues that are in Lawrence Park within a radius of 500 meters

In [25]:
CLIENT_ID = 'ZLUHJQIOX2C1F5UKHGH3EEMT0INXO1KFIDDBQ3J2WHSA5YNS'
CLIENT_SECRET = '553IUYNKFX25TXCKCR1NWS2QEBYRXUHVEXWKBLRSEHZ3QQDW' 
VERSION = '20190605'

LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZLUHJQIOX2C1F5UKHGH3EEMT0INXO1KFIDDBQ3J2WHSA5YNS&client_secret=553IUYNKFX25TXCKCR1NWS2QEBYRXUHVEXWKBLRSEHZ3QQDW&v=20190605&ll=43.7280205,-79.3887901&radius=500&limit=10'

#### Use the URL generated above and Send the GET request and examine the resutls

In [28]:
import requests # library to handle requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d218ef4dbde110025609f1d'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head())
print('\n')
print('Only {} venues were returned by Foursquare'.format(nearby_venues.shape[0]), 'within 500 meters from ', neighborhood_name)

                             name   categories        lat        lng
0            Lawrence Park Ravine         Park  43.726963 -79.394382
1              Zodiac Swim School  Swim School  43.728532 -79.382860
2  TTC Bus #162 - Lawrence-Donway     Bus Line  43.728026 -79.382805


Only 3 venues were returned by Foursquare within 500 meters from  Lawrence Park


## Explore Neighborhoods in Toronto

#### Function to get neaby venues of all the neighborhoods in Toronto

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [43]:
print(toronto_venues.shape)
toronto_venues.head()

(339, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [45]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"Brockton,Exhibition Place,Parkdale Village",10,10,10,10,10,10
Business Reply Mail Processing Centre 969 Eastern,10,10,10,10,10,10
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",10,10,10,10,10,10
"Cabbagetown,St. James Town",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
"Chinatown,Grange Park,Kensington Market",10,10,10,10,10,10
Christie,10,10,10,10,10,10


## Analyze Each Toronto Neighborhood

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
toronto_onehot.shape

(339, 117)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,...,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0.0,0.0
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.1,0.0,0.0,0.00,0.1,0.0,0.0
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,"Chinatown,Grange Park,Kensington Market",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.0
8,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.1,0.0,0.0,0.0,0.1,0.00,0.0,0.0,0.0


In [49]:
toronto_grouped.shape

(38, 117)

#### Print each neighborhood along with the top 5 most common venues

In [50]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                           venue  freq
0               Greek Restaurant   0.1
1                     Food Court   0.1
2  Vegetarian / Vegan Restaurant   0.1
3                      Speakeasy   0.1
4                   Concert Hall   0.1


----Berczy Park----
                           venue  freq
0                   Concert Hall   0.1
1                     Steakhouse   0.1
2  Vegetarian / Vegan Restaurant   0.1
3                           Park   0.1
4                   Cocktail Bar   0.1


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop   0.2
1          Breakfast Spot   0.1
2  Furniture / Home Store   0.1
3                     Bar   0.1
4                    Café   0.1


----Business Reply Mail Processing Centre 969 Eastern----
            venue  freq
0         Brewery   0.1
1   Auto Workshop   0.1
2      Comic Shop   0.1
3      Restaurant   0.1
4  Farmers Market   0.1


----CN Tower,Bathurst Quay,I

#### Sort the venues in descending order and put them into a new DataFrame

In [52]:
## function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
# Create the new dataframe and display the top 10 venues for each neighborhood.

import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Hotel,Concert Hall,Seafood Restaurant,Speakeasy,Plaza,Steakhouse,Asian Restaurant,Greek Restaurant,Food Court,Vegetarian / Vegan Restaurant
1,Berczy Park,Cocktail Bar,Steakhouse,Vegetarian / Vegan Restaurant,French Restaurant,Park,Thai Restaurant,Museum,Farmers Market,Concert Hall,Liquor Store
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Gym,Bar,Italian Restaurant,Café,Caribbean Restaurant,Pet Store,Furniture / Home Store,Breakfast Spot,Concert Hall
3,Business Reply Mail Processing Centre 969 Eastern,Burrito Place,Brewery,Farmers Market,Fast Food Restaurant,Restaurant,Auto Workshop,Skate Park,Pizza Place,Garden Center,Comic Shop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Bar,Airport,Airport Food Court,Airport Gate,Airport Terminal,Coffee Shop,Plane,Harbor / Marina,Wine Bar


## Cluster Neighborhoods

#### Run *k*-means to cluster the neighborhood into 5 clusters

In [63]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 4, 4, 0, 4, 0, 0, 0])

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [67]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Other Great Outdoors,Trail,Pub,Health Food Store,Wine Bar,Dog Run,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,4,Greek Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Italian Restaurant,Pub,Brewery,Concert Hall,Creperie,Eastern European Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,4,Italian Restaurant,Sushi Restaurant,Pub,Liquor Store,Burger Joint,Brewery,Ice Cream Shop,Fish & Chips Shop,Park,Gym
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Bookstore,Ice Cream Shop,Chinese Restaurant,Cheese Shop,Café,Sandwich Place,Comfort Food Restaurant,Fish Market,Coffee Shop,Food
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Eastern European Restaurant,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie


### Finally, let's visualize the resulting clusters

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters